In [1]:
import getpass
import os
from langchain_core.messages import HumanMessage
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage

In [4]:
if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

model = init_chat_model("llama3-8b-8192", model_provider="groq")



response = model.invoke([
    HumanMessage(content="Hi! I'm Bob"),
    AIMessage(content="Hi Bob! It's nice to meet you."),
    HumanMessage(content="What is my name?"),
])
print(response)

content='Your name is Bob!' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 40, 'total_tokens': 46, 'completion_time': 0.005669992, 'prompt_time': 0.009461537, 'queue_time': 0.38616371099999997, 'total_time': 0.015131529}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--25815a36-a958-4ca5-9279-ca9aaebbd9ce-0' usage_metadata={'input_tokens': 40, 'output_tokens': 6, 'total_tokens': 46}


In [7]:
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver

workflow = StateGraph(state_schema=MessagesState)

In [8]:
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages":response}


In [9]:
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")
workflow.add_edge("model", END)


In [10]:
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [11]:
config = {"configurable":{"thread_id":"abc123"}}

In [13]:
query = "I am mayukh"
input_message = [HumanMessage(query)]
output = app.invoke({"messages":input_message}, config)


In [16]:
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

Nice to meet you, Mayukh! Is there something I can help you with or would you like to chat about something in particular?


In [17]:
query = "what is my name?"
input_message = [HumanMessage(query)]
output = app.invoke({"messages":input_message}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Mayukh!
